<a href="https://colab.research.google.com/github/suriya43426/SuperAI_-_Convolutional_Neural_Network/blob/master/revolutionize_forensic_handwriting_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # AI are Revolutionize forensic handwriting analysis  based on generative adversarial networks. (GANs)

### Handwritten signature has been an important identity-verification method since ancient times. Compared with manual handwriting verification, the use of computer image recognition technology for handwriting verification is faster and avoids subjectivity. However, there are still some challenges in traditional image recognition methods, such as feature selection, lack of a standard basis, and low accuracy. For the first time, generative adversarial nets (GAN) technology is adopted to study the task of handwritten signature identification. A special network SIGAN (Signature Identification GAN, SIGAN)is proposed based on the idea of dual learning. The loss value of the trained discriminator of SIGAN is used as the identification threshold. The authenticity of the test handwritten signature is determined by comparing the threshold and loss value of the test image obtained through the network. The experimental data set in this study consists of five hard pen-type signatures, which include some real signatures and some deliberate imitations.

![](https://zefort.com/wp-content/uploads/2018/11/signatures.jpg)

# GANs
### Generative Adversarial Networks, or GANs, are deep learning architecture generative models that have seen wide success.There are thousands of papers on GANs and many hundreds of named-GANs, that is, models with a defined name that often includes “GAN“, such as DCGAN, as opposed to a minor extension to the method. Given the vast size of the GAN literature and number of models, it can be, at the very least, confusing and frustrating as to know what GAN models to focus on. In this post, you will discover the Generative Adversarial Network models that you need to know to establish a useful and productive foundation in the field.

![](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/05/Example-of-the-Architecture-for-the-Stacked-Generative-Adversarial-Network-for-Text-to-Image-Generation-1024x462.png)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate
from keras.optimizers import Adam
from keras.models import load_model, model_from_json

import random
#from PIL import Image, ImageChops

In [ ]:
gen="../input/handwritten-signatures/sample_signature/sample_Signature/genuine"
forg="../input/handwritten-signatures/sample_signature/sample_Signature/forged"

gentr="../input/sigcomp-2009-train/sigcomp 2009 train/Sigcomp 2009 train/genuine"
forgtr="../input/sigcomp-2009-train/sigcomp 2009 train/Sigcomp 2009 train/forgeries"

gent="../input/sigcomp-2009/sigcomp 2009/genuines"
forgt="../input/sigcomp-2009/sigcomp 2009/forgeries"

In [ ]:
img_width, img_height, channels = 224, 224, 3

dim = (img_width, img_height)

def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path,img):
    image=cv2.imread(path+"/"+ img)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image     

def getfiles(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),n,m

def getfiles2(num):
    a=os.listdir(gentr)
    b=os.listdir(forgtr)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    n,m=[],[]
    for i in b:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            n=n+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            n=n+[i]
    for i in a:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            m=m+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            m=m+[i]
    return m.pop(),n,m



In [ ]:
def triplet_loss(y_true, y_pred):
    alpha = 0.5 #whatt is the alpha
    anchor, positive, negative =y_pred[0,0:512], y_pred[0,512:1024], y_pred[0,1024:1536]# why do we take the same cell twice?
    #i mean, 0:512, 512:1024 overlap while 0:511, 512:1023 from the testing file didn't
    
    positive_distance = K.mean(K.square(anchor - positive),axis=-1)
    negative_distance = K.mean(K.square(anchor - negative),axis=-1)
    return K.mean(K.maximum(0.0, positive_distance - negative_distance + alpha))

In [ ]:
model = applications.vgg19.VGG19(weights='imagenet', include_top=False, pooling='max')
#what are these options exactly

In [ ]:
for layer in model.layers[:15]:
    #why are just the top 15 not trainable
    layer.trainable = False

In [ ]:
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

#what exactly are these? I heard about siamese nets, but what about this anchor out ?

anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

In [ ]:
model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [ ]:
model.compile(optimizer=Adam(lr=0.000005),loss=triplet_loss)
#lr is learning rate i think - why this number

In [ ]:
def generator():
    for i in range(1,31):
        if(i<10):
            anc,neg,pos=getfiles(float("0.00"+str(i)),gen,forg)
        else:
            anc,neg,pos=getfiles(float("0.0"+str(i)),gen,forg)
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gen,anc)
                positive=returnimages(gen,pos[j])
                negative=returnimages(forg,neg[i])
               # yield ([anc,pos[j],neg[i]],[0])
                yield ([anchor,positive,negative],[0])

In [ ]:
for x in range(2):
    model.fit_generator(generator(),steps_per_epoch=200,epochs=3)

In [ ]:
model.compile(optimizer=Adam(lr=0.000002),loss=triplet_loss)

In [ ]:
def generator2():
    x=["0.001","0.004", "0.005", "0.006", "0.007",
       "0.008", "0.009", "0.010", "0.011", "0.001", "0.010"]
    #what is this array, why do we take °1 twice (0.001)
  #  x=["0.001", "0.004", "0.006", "0.010"]

    for k in x:
        anc,neg,pos=getfiles2(k)
        frac=0.95 #what is this
        inds = set(random.sample(list(range(len(neg))), int(frac*len(neg))))
        neg = [n for i,n in enumerate(neg) if i not in inds]
    
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gentr,anc)
                positive=returnimages(gentr,pos[j])
                negative=returnimages(forgtr,neg[i])
               # yield ([anc,pos[j],neg[i]])
                yield ([anchor,positive,negative],[0])

In [ ]:
for x in range(2):
    model.fit_generator(generator2(),steps_per_epoch=32,epochs=11)
    
    #why do we fit the model twice

In [ ]:
tneg,tpos=0,0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]
for k in x: #the id of signatures you want to check
    #print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    tneg=tneg+len(neg)
    tpos=tpos+len(pos)
print(tneg,tpos)

In [ ]:
os.listdir('../working')

In [ ]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
#with open('model_architecture.json', 'w') as f:
#    f.write(model.to_json())

In [ ]:
os.listdir('../working')

In [ ]:
forg_passed=0
gen_flagged=0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    
    anchor=returnimages(gent,anc)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor,positive,anchor])
        a, p, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #what are the useless parameters
        #dist=sum(a-p)
        dist=np.linalg.norm(a-p)
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor,negative,anchor])
        a, n, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist=np.linalg.norm(a-n)
        #print("negative distance is ",dist)
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)

In [ ]:
def getfilest(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),m.pop(),n,m

In [ ]:
forg_passed=0
gen_flagged=0
x=[0.002, 0.008, 0.016, 0.018, 0.024, 0.033, 0.035, 0.044, 0.046, 0.063,
   0.070, 0.071, 0.077, 0.084, 0.085, 0.086, 0.089, 0.092, 0.093]

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc1,anc2,neg,pos=getfilest(k,gent,forgt)
    
    anchor1=returnimages(gent,anc1)
    anchor2=returnimages(gent,anc2)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor1,positive,anchor2])
        a1, p, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist1=np.linalg.norm(a1-p)
        dist2=np.linalg.norm(a2-p)
        dist=(dist1+dist2)/2
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor1,negative,anchor2])
        a1, n, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist1=np.linalg.norm(a1-n)
        dist2=np.linalg.norm(a2-n)
        #print("negative distance is ",dist)
        dist=(dist1+dist2)/2
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)